In [ ]:
!pip install bitsandbytes accelerate
!pip install torch transformers
!pip install -U datasets
!pip install -U sentence-transformers

In [1]:
import pandas as pd
df = pd.read_csv('daset.csv')

In [25]:
laliste=[]
laliste_labels=[]
for elt in df['label']:
    laliste_labels.append(elt)
for elt in df['text']:
    laliste.append(elt)

In [3]:
from sentence_transformers import SentenceTransformer
import torch

# Define arguments for 4-bit quantization
model_kwargs = {
    "device_map": "auto",         # Automatically map layers to devices (GPU/CPU)
    "torch_dtype": torch.bfloat16,  # Recommended dtype for Qwen models
    "load_in_4bit": True,           # Enable 4-bit quantization
}

# The model card also recommends enabling flash_attention_2 for better performance
# You may need to install it: pip install flash-attn
# Set use_flash_attention_2=True if available
embedding_model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-8B",
    model_kwargs=model_kwargs,
    trust_remote_code=True, # Often required for custom model architectures
    # use_flash_attention_2=True, # Uncomment if flash-attn is installed
)

/home/ubuntu/synthdata1/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:15<00:00,  3.77s/it]


In [18]:
from datasets import load_dataset
ds = load_dataset("hrithikpiyush/acl-arc")
ds.set_format("pandas")
train=ds["train"][:]
validation=ds["validation"][:]
test=ds["test"][:]

In [21]:

documents = laliste

# Encode the queries and documents. Note that queries benefit from using a prompt
# Here we use the prompt called "query" stored under `model.prompts`, but you can
# also pass your own prompt via the `prompt` argument
synth_data = torch.from_numpy(embedding_model.encode(documents))
validation_data = torch.from_numpy(embedding_model.encode(validation['cleaned_cite_text'].tolist()))
test_data = torch.from_numpy(embedding_model.encode(test['cleaned_cite_text'].tolist()))

In [31]:
import torch

import torch.nn as nn

class MLPClassifier(nn.Module):
    def __init__(self, input_dim=4096, hidden_dim=128, num_classes=6):
        super(MLPClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        return self.model(x)
mymodel = MLPClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mymodel.parameters(), lr=0.001)

In [15]:
mymodel.forward(embeds[0])

tensor([-0.0251, -0.0612, -0.0646, -0.0080, -0.0396, -0.0486],
       grad_fn=<ViewBackward0>)

In [29]:
from torch.utils.data import TensorDataset, DataLoader
X_train = synth_data
y_train = torch.tensor(laliste_labels)

batch_size = 32
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()           # ✅ Reset gradients from previous step
        outputs = mymodel(X_batch)        # Forward pass
        loss = criterion(outputs, y_batch)
        loss.backward()                 # Compute gradients
        optimizer.step()                # Update weights
